# Tutorial 9: Regression Continued

**Regression learning objectives**: 

* Recognize situations where a simple regression analysis would be appropriate for making predictions.
* Explain the $k$-nearest neighbour regression algorithm and describe how it differs from k-nn classification.
* Interpret the output of a $k$-nn regression.
* In a dataset with two variables, perform $k$-nearest neighbour regression in Python using `scikit-learn` to predict the values for a test dataset.
* Execute cross-validation in Python to choose the number of neighbours.
* Using Python, evaluate $k$-nn regression prediction accuracy using a test data set and an appropriate metric (*e.g.*, root means square prediction error).
* In a dataset with > 2 variables, perform $k$-nn regression in Python using `scikit-learn` to predict the values for a test dataset.
* In the context of $k$-nn regression, compare and contrast goodness of fit and prediction properties (namely RMSE vs RMSPE).
* Describe advantages and disadvantages of the $k$-nearest neighbour regression approach.
* Perform ordinary least squares regression in Python using `scikit-learn` to predict the values for a test dataset.
* Compare and contrast predictions obtained from $k$-nearest neighbour regression to those obtained using simple ordinary least squares regression from the same dataset.
* In Python, overlay the ordinary least squares regression lines from `transform_regression` on a single plot. 

In [ ]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor

## Predicting credit card balance

<img src='https://media.giphy.com/media/LCdPNT81vlv3y/giphy-downsized-large.gif' align="left" width='400'>

Source: https://media.giphy.com/media/LCdPNT81vlv3y/giphy-downsized-large.gif

Here in this worksheet we will work with a simulated data set that contains information that we can use to create a model to predict customer credit card balance. A bank might use such information to predict which customers might be the most profitable to lend to (customers who carry a balance, but do not default, for example). Specifically, we wish to build a model to predict credit card balance (`Balance` column) based on income (`Income` column) and credit rating (`Rating` column).

We access this data set by reading it from the `data` folder. Since the first column in the file doesn't have a column name, we should use it as index of the dataframe and then rename it as `ID`.

In [ ]:
credit_original = (
    pd.read_csv("data/Credit.csv", index_col=0)
    .reset_index()
    .rename(columns={"index": "ID"})
)
credit_original.head()

**Question 1.1**
<br> {points: 1}

Select only the columns of data we are interested in using for our prediction (both the predictors and the response variable) using the `.loc[]` function. 

*Note: We could alternatively just leave these variables in and specify them as predictors and response in data splitting steps. But for this worksheet, let's select the relevant columns first.*

*Assign the modified data frame to a variable named `credit` (using a lowercase c).*

In [ ]:
# your code here
raise NotImplementedError
credit

In [ ]:
from hashlib import sha1
assert sha1(str(type(credit)).encode("utf-8")+b"c567a40ae36aa6ff").hexdigest() == "566724260c9b29e335d8257113497ce1b3b71939", "type of type(credit) is not correct"

assert sha1(str(type(credit.shape)).encode("utf-8")+b"217fd12244e5b95a").hexdigest() == "11695032671c4ecd76637e860f61a9e813731b86", "type of credit.shape is not tuple. credit.shape should be a tuple"
assert sha1(str(len(credit.shape)).encode("utf-8")+b"217fd12244e5b95a").hexdigest() == "4e4d680e4fe2a7f762d091480add11624140f4d5", "length of credit.shape is not correct"
assert sha1(str(sorted(map(str, credit.shape))).encode("utf-8")+b"217fd12244e5b95a").hexdigest() == "4fa1f5df796eefa87db3c6977031ae3e1444ff78", "values of credit.shape are not correct"
assert sha1(str(credit.shape).encode("utf-8")+b"217fd12244e5b95a").hexdigest() == "9287c7130dad2466456f1d0d3a934e2ab6a5da00", "order of elements of credit.shape is not correct"

assert sha1(str(type("Income" in credit.columns)).encode("utf-8")+b"7b01c89259e88134").hexdigest() == "4e635241757ae8e2f685d90fadba75d803236a3e", "type of \"Income\" in credit.columns is not bool. \"Income\" in credit.columns should be a bool"
assert sha1(str("Income" in credit.columns).encode("utf-8")+b"7b01c89259e88134").hexdigest() == "3c1178f5d880288d8c291eb8d52ae036d5645380", "boolean value of \"Income\" in credit.columns is not correct"

assert sha1(str(type("Balance" in credit.columns)).encode("utf-8")+b"3ce7697da9fcd538").hexdigest() == "a5e6c4ca521a00f92184732cfd37e106855355f7", "type of \"Balance\" in credit.columns is not bool. \"Balance\" in credit.columns should be a bool"
assert sha1(str("Balance" in credit.columns).encode("utf-8")+b"3ce7697da9fcd538").hexdigest() == "c8f8869ebaa3596bb602a8632cc43f331fdac0b9", "boolean value of \"Balance\" in credit.columns is not correct"

assert sha1(str(type("Rating" in credit.columns)).encode("utf-8")+b"11e15bee81caa932").hexdigest() == "e165b8bc1ebf67749f5f1d3e6c7bc547f63a45d2", "type of \"Rating\" in credit.columns is not bool. \"Rating\" in credit.columns should be a bool"
assert sha1(str("Rating" in credit.columns).encode("utf-8")+b"11e15bee81caa932").hexdigest() == "40e03214504ac287d96f5403eca6075ccbac6c03", "boolean value of \"Rating\" in credit.columns is not correct"

assert sha1(str(type("X1" not in credit.columns)).encode("utf-8")+b"cb1f0e96506865ec").hexdigest() == "ca3f39d083ad2837357c6fa06b2d383c7c545b12", "type of \"X1\" not in credit.columns is not bool. \"X1\" not in credit.columns should be a bool"
assert sha1(str("X1" not in credit.columns).encode("utf-8")+b"cb1f0e96506865ec").hexdigest() == "41d4bf54f9fec42043c11ea10f91736313fde618", "boolean value of \"X1\" not in credit.columns is not correct"

print('Success!')

**Question 1.2**
<br> {points: 1}

**Before** we perform exploratory data analysis, we should create our training and testing data sets. First, split the `credit` data set. Use 60% of the data and assign your answer to objects called `credit_training` and `credit_testing`. Please set the `random_state` as 2000. 

Set the variable we want to predict as the `y_train` and the predictors as `X_train` for `credit_training`. Set the variable we want to predict as the `y_test` and the predictors as `X_test` for `credit_testing`.  

*Assign your answers to variables named `credit_training`, `credit_testing`, `X_train`, `y_train`, `X_test`, and `y_test`.*

In [ ]:
# _____, _____ = train_test_split(
#    ____, ____, random_state=2000           # do not change the random_state
# )

# X_train = ...
# y_train = ...

# X_test = ...
# y_test = ...

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(credit_training is None)).encode("utf-8")+b"d02af853941e4362").hexdigest() == "34425e48765db953639227aa4e07d0c68c1eec7e", "type of credit_training is None is not bool. credit_training is None should be a bool"
assert sha1(str(credit_training is None).encode("utf-8")+b"d02af853941e4362").hexdigest() == "080688b2a40f389bba99d4525c690e25050a161a", "boolean value of credit_training is None is not correct"

assert sha1(str(type(credit_training.shape)).encode("utf-8")+b"f6a580e8eda1203e").hexdigest() == "33690f7a75fd24811738fa6f30497b62a4cb4e7d", "type of credit_training.shape is not tuple. credit_training.shape should be a tuple"
assert sha1(str(len(credit_training.shape)).encode("utf-8")+b"f6a580e8eda1203e").hexdigest() == "168ac5c573a443dc9344b1fa7bc693f6600525d9", "length of credit_training.shape is not correct"
assert sha1(str(sorted(map(str, credit_training.shape))).encode("utf-8")+b"f6a580e8eda1203e").hexdigest() == "22d78ef0750549dc999773ce6f3744628f285213", "values of credit_training.shape are not correct"
assert sha1(str(credit_training.shape).encode("utf-8")+b"f6a580e8eda1203e").hexdigest() == "b0b3fe4849973b6c6095c0c6d81aea5d54e96df5", "order of elements of credit_training.shape is not correct"

assert sha1(str(type(sum(credit_training.Balance))).encode("utf-8")+b"b5bd34a20a3868c4").hexdigest() == "c0e1989b717cfa31b504cd9d904b0d8d5a5211ae", "type of sum(credit_training.Balance) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(credit_training.Balance)).encode("utf-8")+b"b5bd34a20a3868c4").hexdigest() == "393a2e96c33ddc019cdda24da2e8e8e9cf4436ce", "value of sum(credit_training.Balance) is not correct"

assert sha1(str(type(sum(credit_training.Income))).encode("utf-8")+b"1fcd7e227086456a").hexdigest() == "a2a9e6eb8e8c2b8ce1606aa0436501db3a2ba62a", "type of sum(credit_training.Income) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(credit_training.Income), 2)).encode("utf-8")+b"1fcd7e227086456a").hexdigest() == "a03af560a17f69aefa94683c4220e8587cc20d6b", "value of sum(credit_training.Income) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(credit_training.Rating))).encode("utf-8")+b"853cd328eeb8358b").hexdigest() == "9131892c716c4963fe226107bc652373a98cd234", "type of sum(credit_training.Rating) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(credit_training.Rating)).encode("utf-8")+b"853cd328eeb8358b").hexdigest() == "5d5a7d437d655c70b09d255e9f408f0f856c62e1", "value of sum(credit_training.Rating) is not correct"

assert sha1(str(type(credit_testing is None)).encode("utf-8")+b"a52c85174f80143f").hexdigest() == "96e395c7fcbdf2294ce2d8657391e66ff9a6eb8a", "type of credit_testing is None is not bool. credit_testing is None should be a bool"
assert sha1(str(credit_testing is None).encode("utf-8")+b"a52c85174f80143f").hexdigest() == "76e80c20e8458350025e8621ebf1a4272a19dfc3", "boolean value of credit_testing is None is not correct"

assert sha1(str(type(credit_testing.shape)).encode("utf-8")+b"317c43c4ac88c558").hexdigest() == "b2d5fdc353bef2cdc587dee8148f9455710e2c39", "type of credit_testing.shape is not tuple. credit_testing.shape should be a tuple"
assert sha1(str(len(credit_testing.shape)).encode("utf-8")+b"317c43c4ac88c558").hexdigest() == "71baec937f2f38d5deeeff11772a7897bd21a815", "length of credit_testing.shape is not correct"
assert sha1(str(sorted(map(str, credit_testing.shape))).encode("utf-8")+b"317c43c4ac88c558").hexdigest() == "8d286a7cf9596816e10863412315e65d48498008", "values of credit_testing.shape are not correct"
assert sha1(str(credit_testing.shape).encode("utf-8")+b"317c43c4ac88c558").hexdigest() == "5a5d6662f2d593607ecc7f60ca493c8c7ae57ba8", "order of elements of credit_testing.shape is not correct"

assert sha1(str(type(sum(credit_testing.Balance))).encode("utf-8")+b"d08bfd7a7225cb9b").hexdigest() == "5bea01219bfbe5ca9f7f1db7b0073d204e5f7e04", "type of sum(credit_testing.Balance) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(credit_testing.Balance)).encode("utf-8")+b"d08bfd7a7225cb9b").hexdigest() == "191ab1afb07495d6f25bfda89b3c463cd781b62c", "value of sum(credit_testing.Balance) is not correct"

assert sha1(str(type(sum(credit_testing.Income))).encode("utf-8")+b"2ec2853f0ffac31b").hexdigest() == "5800f9a53a4a408a212de89d0eed490ebe9b8cd2", "type of sum(credit_testing.Income) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(credit_testing.Income), 2)).encode("utf-8")+b"2ec2853f0ffac31b").hexdigest() == "e7074e1a344efd5af956929b2cb73ad8fb72608a", "value of sum(credit_testing.Income) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(credit_testing.Rating))).encode("utf-8")+b"45640638ff41cd19").hexdigest() == "e6ebbb2d00cb3e73c587f3edda424b9a5b56ed8c", "type of sum(credit_testing.Rating) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(credit_testing.Rating)).encode("utf-8")+b"45640638ff41cd19").hexdigest() == "f1f1e6c425b2ca96c3d2a96bd101442221483a1c", "value of sum(credit_testing.Rating) is not correct"

assert sha1(str(type(X_train.columns.values)).encode("utf-8")+b"add2211afd02100f").hexdigest() == "462b3f24d358f3a88fdb250ff6cf7bf67f09029b", "type of X_train.columns.values is not correct"
assert sha1(str(X_train.columns.values).encode("utf-8")+b"add2211afd02100f").hexdigest() == "40109dcc68d878195d5b4abff00edd7fa0508c15", "value of X_train.columns.values is not correct"

assert sha1(str(type(X_train.shape)).encode("utf-8")+b"e81cdfba924e206c").hexdigest() == "1717503307b529068f22603e390f0854c74cc2f1", "type of X_train.shape is not tuple. X_train.shape should be a tuple"
assert sha1(str(len(X_train.shape)).encode("utf-8")+b"e81cdfba924e206c").hexdigest() == "2ae06f9bf3a391dfe2e45a60afab2f3102ef9a3d", "length of X_train.shape is not correct"
assert sha1(str(sorted(map(str, X_train.shape))).encode("utf-8")+b"e81cdfba924e206c").hexdigest() == "ae280d1231c10decb317c93004c3e952a72ce298", "values of X_train.shape are not correct"
assert sha1(str(X_train.shape).encode("utf-8")+b"e81cdfba924e206c").hexdigest() == "5d555539f9ff712dfb423315395f6153304d3c32", "order of elements of X_train.shape is not correct"

assert sha1(str(type(y_train.name)).encode("utf-8")+b"ded0af6a1e70f078").hexdigest() == "769f17f599999e391aabcaf771bc094ea5b9682d", "type of y_train.name is not str. y_train.name should be an str"
assert sha1(str(len(y_train.name)).encode("utf-8")+b"ded0af6a1e70f078").hexdigest() == "5b78ff050fac19764d53e49228fe853ef0444d14", "length of y_train.name is not correct"
assert sha1(str(y_train.name.lower()).encode("utf-8")+b"ded0af6a1e70f078").hexdigest() == "f0615cc3772ea02186385ae4728a5b924e43c8fe", "value of y_train.name is not correct"
assert sha1(str(y_train.name).encode("utf-8")+b"ded0af6a1e70f078").hexdigest() == "9142d1425f673a4687ebdd96239c220fc1afc0b2", "correct string value of y_train.name but incorrect case of letters"

assert sha1(str(type(y_train.shape)).encode("utf-8")+b"e53beb670564c634").hexdigest() == "4c44b0a5ea29cd3c67bc12de92edd5bd4be94f8d", "type of y_train.shape is not tuple. y_train.shape should be a tuple"
assert sha1(str(len(y_train.shape)).encode("utf-8")+b"e53beb670564c634").hexdigest() == "5d0edc3c56c18b02eed55116c271647386a2de45", "length of y_train.shape is not correct"
assert sha1(str(sorted(map(str, y_train.shape))).encode("utf-8")+b"e53beb670564c634").hexdigest() == "b5997745d9fa60d31cf1c7999d267d55013a9356", "values of y_train.shape are not correct"
assert sha1(str(y_train.shape).encode("utf-8")+b"e53beb670564c634").hexdigest() == "abc35ecd1442ec11b6a0bb71c0989dddf6f6064c", "order of elements of y_train.shape is not correct"

assert sha1(str(type(X_test.columns.values)).encode("utf-8")+b"054a59701befc51f").hexdigest() == "c951d7122206c3302d0712a28ecf1dee8ffd584d", "type of X_test.columns.values is not correct"
assert sha1(str(X_test.columns.values).encode("utf-8")+b"054a59701befc51f").hexdigest() == "401c6ecd49635263e9033d1837cdceeb4df064ee", "value of X_test.columns.values is not correct"

assert sha1(str(type(X_test.shape)).encode("utf-8")+b"47327772648ee6d6").hexdigest() == "174f102ebe10475c6b4735d500e75c5afe6f6929", "type of X_test.shape is not tuple. X_test.shape should be a tuple"
assert sha1(str(len(X_test.shape)).encode("utf-8")+b"47327772648ee6d6").hexdigest() == "a84fb4b85e0ebfd93453f4b71e5a05232fba4c7e", "length of X_test.shape is not correct"
assert sha1(str(sorted(map(str, X_test.shape))).encode("utf-8")+b"47327772648ee6d6").hexdigest() == "a12840f56ec240d0e3d05a1cb95857ebe8bdf766", "values of X_test.shape are not correct"
assert sha1(str(X_test.shape).encode("utf-8")+b"47327772648ee6d6").hexdigest() == "4c0aa170e8d4ca6f95c5642bfee2d45fc66b8539", "order of elements of X_test.shape is not correct"

assert sha1(str(type(y_test.name)).encode("utf-8")+b"d568746a6820a016").hexdigest() == "cf0fd04136bece90025fca345af7cd2af2152043", "type of y_test.name is not str. y_test.name should be an str"
assert sha1(str(len(y_test.name)).encode("utf-8")+b"d568746a6820a016").hexdigest() == "12a0747fc1ad363baa34cf658fc3d3564eb645b1", "length of y_test.name is not correct"
assert sha1(str(y_test.name.lower()).encode("utf-8")+b"d568746a6820a016").hexdigest() == "ef8a1805a2841135426894f74730442a14cd8764", "value of y_test.name is not correct"
assert sha1(str(y_test.name).encode("utf-8")+b"d568746a6820a016").hexdigest() == "8f11c9a51a2c36156821424bce4cddb7e0f2ce70", "correct string value of y_test.name but incorrect case of letters"

assert sha1(str(type(y_test.shape)).encode("utf-8")+b"49c6ece4df6c9d35").hexdigest() == "adf4225359f368ad19b4118d871ede61ee91bb7f", "type of y_test.shape is not tuple. y_test.shape should be a tuple"
assert sha1(str(len(y_test.shape)).encode("utf-8")+b"49c6ece4df6c9d35").hexdigest() == "3f9ad75410b15930f3dfc597111b3e9dea34fc62", "length of y_test.shape is not correct"
assert sha1(str(sorted(map(str, y_test.shape))).encode("utf-8")+b"49c6ece4df6c9d35").hexdigest() == "74ffa0fdc4934ce22a2f0e5960710bac28b7e74b", "values of y_test.shape are not correct"
assert sha1(str(y_test.shape).encode("utf-8")+b"49c6ece4df6c9d35").hexdigest() == "ec01fb92b4565f520abb4775c1b8444a0c405d71", "order of elements of y_test.shape is not correct"

print('Success!')

**Question 1.3**
<br> {points: 1}

Using only the observations in the training data set, create a scatter plot matrix (or pairplot) of all the columns we are interested in including in our model. 

*Name the plot object `credit_eda`.*

In [ ]:
# your code here
raise NotImplementedError
credit_eda

In [ ]:
from hashlib import sha1
assert sha1(str(type(credit_eda is None)).encode("utf-8")+b"865277805bac1aa2").hexdigest() == "8dab9d1bd97ee2b8c70e37c3b028e58f80dab20d", "type of credit_eda is None is not bool. credit_eda is None should be a bool"
assert sha1(str(credit_eda is None).encode("utf-8")+b"865277805bac1aa2").hexdigest() == "aa07690418f7675bda482db542e7f495f0158763", "boolean value of credit_eda is None is not correct"

assert sha1(str(type(credit_eda)).encode("utf-8")+b"c2b46c28842e4c6f").hexdigest() == "e303c2dcaf5444de05ec4b6bd2514ec6bcb214f0", "type of type(credit_eda) is not correct"

print('Success!')

**Question 1.4**
<br> {points: 1} 

Looking at the pairplot above, which of the following statements is **incorrect**?

A. There is a strong positive relationship between the response variable (`Balance`) and the `Rating` predictor

B. There is a strong positive relationship between the two predictors (`Income` and `Rating`)

C. There is a strong positive relationship between the response variable (`Balance`) and the `Income` predictor

D. None of the above statements are incorrect

*Assign your answer to an object called `answer1_4`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer1_4

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_4)).encode("utf-8")+b"779b2bbbe29ffa25").hexdigest() == "ff4b4edb17f3db618832cc7937566ca1a61c7df7", "type of answer1_4 is not str. answer1_4 should be an str"
assert sha1(str(len(answer1_4)).encode("utf-8")+b"779b2bbbe29ffa25").hexdigest() == "404af28d6c1217326125e09aed7098181806cef2", "length of answer1_4 is not correct"
assert sha1(str(answer1_4.lower()).encode("utf-8")+b"779b2bbbe29ffa25").hexdigest() == "f43f70ffa54b8f4dd4229476bb683664ea291f54", "value of answer1_4 is not correct"
assert sha1(str(answer1_4).encode("utf-8")+b"779b2bbbe29ffa25").hexdigest() == "6da28e6d16ceed289b21a7cc4573681f99c0ed04", "correct string value of answer1_4 but incorrect case of letters"

print('Success!')

**Question 1.5**
<br> {points: 1}

Now that we have our training data, we will fit a linear regression model. Create a `LinearRegression` object.

*Assign the object to a variable named `lm`.*

In [ ]:
# lm = ___()

# your code here
raise NotImplementedError
print(lm)

In [ ]:
from hashlib import sha1
assert sha1(str(type(lm is None)).encode("utf-8")+b"0f5ad1158a213775").hexdigest() == "21fed1295e942a0b59a697bebd02456c1686015e", "type of lm is None is not bool. lm is None should be a bool"
assert sha1(str(lm is None).encode("utf-8")+b"0f5ad1158a213775").hexdigest() == "a46a6d36918d30f24c95f70e3dbc63dae40d35ce", "boolean value of lm is None is not correct"

assert sha1(str(type(type(lm))).encode("utf-8")+b"9d69b73925e66e22").hexdigest() == "f91891b3766bbb6e42481d61457029c3f5c77e6f", "type of type(lm) is not correct"
assert sha1(str(type(lm)).encode("utf-8")+b"9d69b73925e66e22").hexdigest() == "7cd16fccd54dac6de6abe05f6f597e03a30e5f5c", "value of type(lm) is not correct"

print('Success!')

**Question 1.6**
<br> {points: 1}

Now that we have our model, fit our simple linear regression model. 

*Assign the fit to an object called `credit_fit`.*

In [ ]:
# your code here
raise NotImplementedError
print(credit_fit.coef_)
print(credit_fit.intercept_)

In [ ]:
from hashlib import sha1
assert sha1(str(type(credit_fit is None)).encode("utf-8")+b"11c3d40a4e4aca33").hexdigest() == "4062161e181a9d0ae2164819b9ef0b3bb21ff8d7", "type of credit_fit is None is not bool. credit_fit is None should be a bool"
assert sha1(str(credit_fit is None).encode("utf-8")+b"11c3d40a4e4aca33").hexdigest() == "65e9be04b01737bfbd7141f2fcfdf56d4093d003", "boolean value of credit_fit is None is not correct"

assert sha1(str(type(type(credit_fit))).encode("utf-8")+b"ade9198db4a502aa").hexdigest() == "a6aaf177d1e9c6156ec87d002fd9243598a45604", "type of type(credit_fit) is not correct"
assert sha1(str(type(credit_fit)).encode("utf-8")+b"ade9198db4a502aa").hexdigest() == "7c986cbb5d193aa2ebb3d7af630fba5ccb9ad7c1", "value of type(credit_fit) is not correct"

assert sha1(str(type(round(credit_fit.coef_[0], 2))).encode("utf-8")+b"0835df810275e08c").hexdigest() == "d9a21b9e05c0782641b44e47064049901272ef63", "type of round(credit_fit.coef_[0], 2) is not correct"
assert sha1(str(round(credit_fit.coef_[0], 2)).encode("utf-8")+b"0835df810275e08c").hexdigest() == "6dc00c4d913b58efb4509a86d8d82ec60ae8f48c", "value of round(credit_fit.coef_[0], 2) is not correct"

assert sha1(str(type(round(credit_fit.coef_[1], 2))).encode("utf-8")+b"46c87b0cf4b7d91e").hexdigest() == "fcc3120d5a9c7a126a6340e8c731218a48a9bce2", "type of round(credit_fit.coef_[1], 2) is not correct"
assert sha1(str(round(credit_fit.coef_[1], 2)).encode("utf-8")+b"46c87b0cf4b7d91e").hexdigest() == "b18107598ec9a6da4550da89903cb3bc9ab19a2c", "value of round(credit_fit.coef_[1], 2) is not correct"

assert sha1(str(type(round(credit_fit.fit_intercept, 2))).encode("utf-8")+b"691c60bea45f4a57").hexdigest() == "6b35f9884a594231c87d78dcb9fb1c4498273799", "type of round(credit_fit.fit_intercept, 2) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(round(credit_fit.fit_intercept, 2)).encode("utf-8")+b"691c60bea45f4a57").hexdigest() == "d07048b45ca5ec4ce9c96edca4d2416d3bfb0f12", "value of round(credit_fit.fit_intercept, 2) is not correct"

assert sha1(str(type(credit_fit.n_features_in_)).encode("utf-8")+b"f441cf7a75990acb").hexdigest() == "57d73b43791c73043e3630f0c27d5f9a922abb09", "type of credit_fit.n_features_in_ is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(credit_fit.n_features_in_).encode("utf-8")+b"f441cf7a75990acb").hexdigest() == "197f78de065e98f8912d2077412ed7111e4255ad", "value of credit_fit.n_features_in_ is not correct"

print('Success!')

**Question 1.7**
<br> {points: 1}

Examine the `credit_fit` to obtain the slopes/coefficients for each predictor and intercept. Which of the following mathematical equations is correct for your prediction model?

A. $credit\: card \: balance = -530.674 -7.190*income  + 3.914*credit\: card\: rating$

B. $credit\: card \: balance = -530.674 + 3.914*income  -7.190*credit\: card\: rating$

C. $credit\: card \: balance = 530.674 -7.190*income  - 3.914*credit\: card\: rating$

D. $credit\: card \: balance = 530.674 - 3.914*income  + 7.190*credit\: card\: rating$

*Assign your answer to an object called `answer1_7`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer1_7

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_7)).encode("utf-8")+b"28dab8ac13e95833").hexdigest() == "0d645c2a693aab258294816903b69bfbafd09e06", "type of answer1_7 is not str. answer1_7 should be an str"
assert sha1(str(len(answer1_7)).encode("utf-8")+b"28dab8ac13e95833").hexdigest() == "98922eed273d982b60ac060da483aff91c3d6398", "length of answer1_7 is not correct"
assert sha1(str(answer1_7.lower()).encode("utf-8")+b"28dab8ac13e95833").hexdigest() == "74de9198b0d76cc66500d452e912f1e173fc38a9", "value of answer1_7 is not correct"
assert sha1(str(answer1_7).encode("utf-8")+b"28dab8ac13e95833").hexdigest() == "3d421ddac8434c32fa7c4d79aee95dd31674c61c", "correct string value of answer1_7 but incorrect case of letters"

print('Success!')

**Question 1.8**
<br> {points: 1}

Evaluate the RMSE to assess goodness of fit on `credit_fit` (remember this is how well it predicts on the **training data** used to fit the model). First, calculate the predicted values for the test data and saved it in an object called `credit_predictions`. Next, use the `mean_squared_error` function to calculate the mean squared error between the predicted value and the training response variables. Finally, calculate the RMSE .

*Assign your answer to a single numerical value called `lm_rmse`.*

In [ ]:
# your code here
raise NotImplementedError
lm_rmse

In [ ]:
from hashlib import sha1
assert sha1(str(type(lm_rmse)).encode("utf-8")+b"409e04a0eda2353e").hexdigest() == "c402f36eb1706c9ee72d6418ad02666a59198bcd", "type of lm_rmse is not correct"
assert sha1(str(lm_rmse).encode("utf-8")+b"409e04a0eda2353e").hexdigest() == "282e1be9984080334761dddf8ad42324e4708192", "value of lm_rmse is not correct"

print('Success!')

**Question 1.9**
<br> {points: 1}

Now evaluate the RMSPE on the test data. Take the same steps as before, but compare the model's predictions and actual response variable values on the **test data.**

*Assign your answer to a single numerical value called `lm_rmspe`.*

In [ ]:
# your code here
raise NotImplementedError
lm_rmse

In [ ]:
from hashlib import sha1
assert sha1(str(type(lm_rmse)).encode("utf-8")+b"2478befe19b6cb49").hexdigest() == "ecf0a6f6b5885ecdc583bc7c69cb61db9e2b8ca7", "type of lm_rmse is not correct"
assert sha1(str(lm_rmse).encode("utf-8")+b"2478befe19b6cb49").hexdigest() == "305b72e607b1902e272e7af9cf95b8bdd6f0bba6", "value of lm_rmse is not correct"

print('Success!')

**Question 1.9.1**
<br> {points: 3}

Redo this analysis using $k$-nn regression instead of linear regression. Use the same predictors and train - test data splits as you used for linear regression. If you need help, follow the step-by-step instructions below. 

1. Create the $k$-nn regression model called `credit_knn`.

2. Create a parameter grid for `n_neighbors`. Assign the parameter grid to an object called `gridvals`. 

3. Tune the # of neighbors using `GridSearchCV` for 5 cross validations and use `neg_root_mean_squared_error` as scoring method. Assign your answer to an object called `credit_tuned`.

4. Fit your tuned model to `X_train` and `y_train`. 

5.  From your results, find the lowest value of RMSPE (the mean cross-validated RMSPE of the best estimator) and assign it to a single numerical value called `knn_rmspe`. Remember to add a negative sign `-` to the best score output by the cross validation, as it uses negative RMSPE internally!

In [ ]:
np.random.seed(2020) # Do not change!

# your code here
raise NotImplementedError
knn_rmspe

**Question 1.9.2** 
<br> {points: 3}

Discuss which model, linear regression versus $k$-nn regression, gives better predictions and why you think that might be happening.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 2. Ames Housing Prices

<img src="https://media.giphy.com/media/xUPGGuzpmG3jfeYWIg/giphy.gif" width = "600"/>

Source: https://media.giphy.com/media/xUPGGuzpmG3jfeYWIg/giphy.gif

If we take a look at the Business Insider report [What do millenials want in a home?](https://www.businessinsider.com/what-do-millennials-want-in-a-home-2017-2), we can see that millenials like newer houses that have their own defined spaces. Today we are going to be looking at housing data to understand how the sale price of a house is determined. Finding highly detailed housing data with the final sale prices is very hard, however researchers from Truman State Univeristy have studied and made available a dataset containing multiple variables for the city of Ames, Iowa. The data set describes the sale of individual residential property in Ames, Iowa
from 2006 to 2010. You can read more about the data set [here](http://jse.amstat.org/v19n3/decock.pdf). Today we will be looking at 5 different variables to predict the sale price of a house. These variables are: 

- Lot Area: `lot_area`
- Year Built: `year_built`
- Basement Square Footage: `bsmt_sf`
- First Floor Square Footage: `first_sf`
- Second Floor Square Footage: `second_sf`

First, load the data with the script given below. 

In [ ]:
# run this cell

ames_data = (
    pd.read_csv("data/ames.csv")
    .loc[
        :,
        [
            "Lot.Area",
            "Year.Built",
            "Total.Bsmt.SF",
            "X1st.Flr.SF",
            "X2nd.Flr.SF",
            "SalePrice",
        ],
    ]
    .rename(
        columns={
            "Lot.Area": "lot_area",
            "Year.Built": "year_built",
            "Total.Bsmt.SF": "bsmt_sf",
            "X1st.Flr.SF": "first_sf",
            "X2nd.Flr.SF": "second_sf",
            "SalePrice": "sale_price",
        }
    )
    .dropna()
)

ames_data

**Question 2.1**
<br> {points: 3}

Split the data into a train dataset and a test dataset, based on a 70%-30% train-test split. Use `random_state = 2019`. Remember that we want to predict the `sale_price` based on all of the other variables. 

*Assign the objects to `ames_training`, `ames_testing`, `X_train_ames`, `y_train_ames`, `X_test_ames` and `y_test_ames` respectively.*

In [ ]:
# ___, ___ = train_test_split(
#     ___, ___, random_state=2019  # do not change the random state
# )

# X_train_ames = ...
# y_train_ames = ...
# X_test_ames = ...
# y_test_ames = ...


# your code here
raise NotImplementedError

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

X_train_ames
y_train_ames
X_test_ames
y_test_ames

print('Success!')

**Question 2.2**
<br> {points: 3}

Let's start by exploring the training data. Create a scatter plot matrix (or pairplot) of all the columns to explore the relationships between the different variables. 

*Assign your plot object to a variable named `answer2_2`.*

In [ ]:
# your code here
raise NotImplementedError
answer2_2

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

answer2_2

print('Success!')

**Question 2.3**
<br> {points: 1}

Now that we have seen all the relationships between the variables, which of the following variables would be the strongest predictor for `sale_price`?

A. `bsmt_sf`

B. `year_built`

C. `first_sf`

D. `lot_area`

E. `second_sf`

F. It isn't clear from these plots

*Assign your answer to an object called `answer2_3`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_3

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

answer2_3

print('Success!')

**Question 2.4**
<br> {points: 3}

Fit a linear regression model from `scikit-learn` with `X_train_ames` and `y_train_ames`. 

- create a linear regression model called `lm`
- fit the model to `X_train_ames` and `y_train_ames` and name it as `ames_fit`

*Save the fit to an object called `ames_fit`.*

In [ ]:
# your code here
raise NotImplementedError
ames_fit

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

lm
ames_fit

print('Success!')

**Question 2.5**
<br> {points: 1}

True or false: "Aside from the intercept, all the variables have a positive relationship with the `sale_price`. As the value of the variables decrease, the prices of the houses increase."

*Assign your answer to an object called `answer2_5`. Make sure your answer is either `True` or `False`.*

In [ ]:
# your code here
raise NotImplementedError
answer2_5

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

answer2_5


print('Success!')

In [ ]:
# run this cell
print(f"coefficients: {ames_fit.coef_.tolist()}")
print(f"intercept: {ames_fit.intercept_}")

**Question 2.6**
<br> {points: 3}

Looking at the coefficients and intercept produced from the cell block above, write down the equation for the linear model.

Make sure to use correct math typesetting syntax (i.e., surround your answer with dollar signs, $a = b$)

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.


**Question 2.7**
<br> {points: 1}

Why can we not easily visualize the model above as a line or a plane in a single plot?

A. This is not true, we can actually easily visualize the model

B. The intercept is much larger (6 digits) than the coefficients (single/double digits)

C. There are more than 2 predictors

D. None of the above

*Assign your answer to an object called `answer2_7`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_7

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

answer2_7

print('Success!')

**Question 2.8**
<br> {points: 3}

We need to evaluate how well our model is doing. For this question, calculate the RMSPE (a single numerical value) of the linear regression model using the test data set.

*Assign your answer to an object named `ames_rmspe`.*

In [ ]:
np.random.seed(2020) # DO NOT REMOVE

# ames_predictions = ...
# ames_rmspe = ...

# your code here
raise NotImplementedError
ames_rmspe

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

ames_rmspe

print('Success!')

**Question 2.9**
<br> {points: 1}

Which of the following statements is **incorrect**?

A. RMSE is a measure of goodness of fit 

B. RMSE measures how well the model predicts on data it was trained with 

C. RMSPE measures how well the model predicts on data it was not trained with 

D. RMSPE measures how well the model predicts on data it was trained with

*Assign your answer to an object called `answer2_9`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_9

In [ ]:
# Note: this question has hidden tests! You can't see these while you're working.
# We list the objects you must create here to make sure you actually created them.

answer2_9

print('Success!')